In [1]:
import re
from bs4 import BeautifulSoup as bs
from splinter import Browser
import pandas as pd
import requests
from selenium import webdriver

In [2]:
executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
#executable_path = {"executable_path": "c:/bin/chromedriver.exe"}
browser = Browser("chrome", **executable_path, headless=True)
url = 'https://mars.nasa.gov/news/'
browser.visit(url)
html = browser.html
soup = bs(html)

In [3]:
#NASA Mars News
news_title = soup.find('div', class_='content_title').get_text()
print(news_title)
news_paragraph = soup.find('div', class_='article_teaser_body').get_text()
print(news_paragraph)

Mars Now
In honor of Hispanic Heritage Month, Christina Hernandez, an instrument engineer on the Mars 2020 mission, talks about her childhood and journey to NASA.


In [4]:
# JPL Mars Space Images - Featured Image 
def init_browser():
    executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
#    executable_path = {"executable_path": "C:/bin/chromedriver.exe"}
    return Browser("chrome", **executable_path, headless=False)

In [5]:
# Run init_browser/driver.
browser = init_browser()

# Visit the url for JPL Featured Space Image.
jpl_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(jpl_url)

# Select "FULL IMAGE".
browser.click_link_by_partial_text("FULL IMAGE")

# Find "more info" for first image, set to variable, and command click.
browser.is_element_present_by_text("more info", wait_time=1)
more_info_element = browser.find_link_by_partial_text("more info")
more_info_element.click()

# HTML Object.
html = browser.html

# Parse HTML with Beautiful Soup
image_soup = bs(html)

# Scrape image URL.
image_url = image_soup.find("figure", class_="lede").a["href"]

# Concatentate https://www.jpl.nasa.gov with image_url.
featured_image_url = f'https://www.jpl.nasa.gov{image_url}'

# Exit Browser.
browser.quit()

/Users/zhr47/opt/anaconda3/envs/NewPythonData/lib/python3.6/site-packages/splinter/driver/webdriver/__init__.py:493: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  FutureWarning,


In [6]:
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA14944_hires.jpg


In [7]:
# Mars facts
# URL for Mars Facts.
facts_url = "https://space-facts.com/mars/"

# Use Panda's `read_html` to parse the URL.
facts_tables = pd.read_html(facts_url)

# Required table stored in index "0".
# Save as DF.
df_mars_facts = facts_tables[0]
#print(df_mars_facts)

# Rename columns.
df_mars_facts.columns = ['Description', 'Value']

# Set index to 'Description'.
df_mars_facts.set_index('Description', inplace=True)

df_mars_facts

,Value
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [9]:
# Convert DF to html and save in Resources Folder.
df_mars_facts.to_html('Resources/mars_facts.html')


In [10]:
# Convert DF to HTML string.
mars_facts = df_mars_facts.to_html(header=True, index=True)
print(mars_facts)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Value</th>
    </tr>
    <tr>
      <th>Description</th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Equatorial Diameter:</th>
      <td>6,792 km</td>
    </tr>
    <tr>
      <th>Polar Diameter:</th>
      <td>6,752 km</td>
    </tr>
    <tr>
      <th>Mass:</th>
      <td>6.39 × 10^23 kg (0.11 Earths)</td>
    </tr>
    <tr>
      <th>Moons:</th>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <th>Orbit Distance:</th>
      <td>227,943,824 km (1.38 AU)</td>
    </tr>
    <tr>
      <th>Orbit Period:</th>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <th>Surface Temperature:</th>
      <td>-87 to -5 °C</td>
    </tr>
    <tr>
      <th>First Record:</th>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <th>Recorded By:</th>
      <td>Egyptian astronomers</td>
    </tr>
  </tbody>
</table>


In [11]:
#Mars Hemispheres
# Run init_browser/driver.
browser = init_browser()

# Visit the url for USGS Astrogeology.
astrogeo_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(astrogeo_url)

# HTML Object.
html = browser.html

# Parse HTML with Beautiful Soup
astrogeo_soup = bs(html)

# Store main URL in a variable so that 'href' can be appended to it after each iteration.
main_astrogeo_url = "https://astrogeology.usgs.gov"

# Each link is located in 'div' tag, class "item".
# Locate all 4 and store in variable.
hems_url = astrogeo_soup.find_all("div", class_="item")

# Create empty list for each Hemisphere URL.
hemis_url = []

for hem in hems_url:
    hem_url = hem.find('a')['href']
    hemis_url.append(hem_url)

browser.quit()


In [12]:
print(hemis_url)

['/search/map/Mars/Viking/cerberus_enhanced', '/search/map/Mars/Viking/schiaparelli_enhanced', '/search/map/Mars/Viking/syrtis_major_enhanced', '/search/map/Mars/Viking/valles_marineris_enhanced']


In [13]:
# Create list of dictionaries called hemisphere_image_urls.
# Iterate through all URLs saved in hemis_url.
# Concatenate each with the main_astrogeo_url.
# Confirm the concat worked properly: confirmed.
# Visit each URL.

hemisphere_image_urls = []
for hemi in hemis_url:
    hem_astrogeo_url = main_astrogeo_url + hemi
    print(hem_astrogeo_url)
    
    # Run init_browser/driver.
    browser = init_browser()
    browser.visit(hem_astrogeo_url)
    
    # HTML Object.
    html = browser.html

    # Parse HTML with Beautiful Soup
    hemi_soup = bs(html)

    # Locate each title and save to raw_title, to be cleaned.
    raw_title = hemi_soup.find("h2", class_="title").text
    
    # Remove ' Enhanced' tag text from each "title" via split on ' Enhanced'.
    title = raw_title.split(' Enhanced')[0]
    
    # Locate each 'full.jpg' for all 4 Hemisphere URLs.
    img_url = hemi_soup.find("li").a['href']
    
    # Append both title and img_url to 'hemisphere_image_url'.
    hemisphere_image_urls.append({'title': title, 'img_url': img_url})
    
    browser.quit()


https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced
https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced
https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced
https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced


In [14]:
print(hemisphere_image_urls)

[{'title': 'Cerberus Hemisphere', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]


In [15]:
#Mars Data Dictionary - MongoDB 

In [16]:
# Create empty dictionary for all Mars Data.
mars_data = {}

# Append news_title and news_paragraph to mars_data.
mars_data['news_title'] = news_title
mars_data['news_paragraph'] = news_paragraph

In [17]:
# Append featured_image_url to mars_data.
mars_data['featured_image_url'] = featured_image_url

In [18]:
# Append mars_facts to mars_data.
mars_data['mars_facts'] = mars_facts

In [19]:
# Append hemisphere_image_urls to mars_data.
mars_data['hemisphere_image_urls'] = hemisphere_image_urls


In [20]:
print("Scrape Complete!!!")

Scrape Complete!!!


In [21]:
# Close the browser after scraping
browser.quit


<bound method BaseWebDriver.quit of <splinter.driver.webdriver.chrome.WebDriver object at 0x7fccc8646898>>

In [22]:
mars_data

{'news_title': 'Mars Now',
 'news_paragraph': 'In honor of Hispanic Heritage Month, Christina Hernandez, an instrument engineer on the Mars 2020 mission, talks about her childhood and journey to NASA.',
 'featured_image_url': 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA14944_hires.jpg',
 'mars_facts': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Value</th>\n    </tr>\n    <tr>\n      <th>Description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Per